In [1]:
import getpass
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import chromadb
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

load_dotenv()

model = "gpt-3.5-turbo"

llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
from langchain_chroma import Chroma
os.environ["LANGSMITH_PROJECT"] = "data-chunking-pdf"

chroma_client = chromadb.HttpClient(host='localhost', port=8000)
print(chroma_client.heartbeat())

vector_store = Chroma(
    collection_name="llm-rag-ai",
    embedding_function=embeddings,
    client=chroma_client
)
retriever = vector_store.as_retriever()


1739319403649052376


In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

template = """
**Context**: {context}
You are an AI assistant specialized in answering questions about Artificial Intelligence (AI), Large Language Models (LLMs), Retrieval-Augmented Generation (RAG), and Python programming. Your task is to provide clear, accurate, and concise explanations. If the question is outside your scope, politely inform the user.

**User Question**: {question}

**Your Response**:
1. Provide a clear and structured answer.
2. Include examples, code snippets, or analogies if applicable.
3. Do not include any additional explanations or context.
4. Do not hallucinate.
"""
prompt_template = ChatPromptTemplate([
    ("system", template),
    ("user", "{question}")
])

In [4]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt_template
        | llm
        | StrOutputParser()
)

response = rag_chain.invoke("give me some use cases for rag")
print(response)

Some use cases for Retrieval-Augmented Generation (RAG) include:

1. **Healthcare**: Generating personalized treatment plans for patients based on their medical history and current health status.
2. **Finance**: Providing investment advice based on market trends and client preferences.
3. **Education**: Generating personalized educational content for students based on their learning needs.
4. **Real Estate**: Analyzing housing prices and demographic information to provide accurate property valuations.
5. **Energy**: Optimizing energy supply and reducing waste in the energy sector.

These are just a few examples of how RAG can be applied across various industries to generate personalized answers and recommendations.


In [5]:
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"],k=3)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt_template.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [7]:
response = graph.invoke({"question": "explain langchain"})
print(response["answer"])

LangChain is a framework for developing applications powered by language models. It enables context-aware applications that connect a language model to various sources of context and allows for reasoning based on the provided context. LangChain provides a digital toolbox to build intelligent applications that can communicate, understand, and make decisions similar to humans. It allows developers to tap into the knowledge of advanced language models like GPT-4, PaLM, Gemini, and open source models such as LLaMA. Additionally, LangChain chains are foundational concepts that help build robust generative AI applications by organizing tasks into interconnected steps within an efficient workflow.
